In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("large_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from utils.config import Config
from utils.parse_dataset import NetworkDataset, load_datasets
from utils.load_models import MLP_Models

In [2]:
conf = Config()
load_models = MLP_Models()
model = load_models.mlp_4
print(conf.large_models_path)
print(conf.datasets)
print(conf.device)

/home/jorgetf/testmodel/Network-Packet-ML-Model/large_models
/home/jorgetf/testmodel/Network-Packet-ML-Model/datasets
cuda


In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

In [4]:
data, labels = next(iter(train_loader))
print(data.shape, labels.shape)

torch.Size([512, 513]) torch.Size([512])


In [5]:
acc, train_loss, val_loss = model.train(train_loader, val_loader, conf.epochs)

Epoch: 1/20, Accuracy: 61.68%, Train loss: 1.3552, Val loss: 1.2681
Epoch: 2/20, Accuracy: 67.41%, Train loss: 0.7765, Val loss: 0.9181
Epoch: 3/20, Accuracy: 66.91%, Train loss: 0.6135, Val loss: 0.8985
Epoch: 4/20, Accuracy: 60.53%, Train loss: 0.5252, Val loss: 0.9102
Epoch: 5/20, Accuracy: 75.64%, Train loss: 0.4736, Val loss: 0.5519
Epoch: 6/20, Accuracy: 73.76%, Train loss: 0.4183, Val loss: 0.6319
Epoch: 7/20, Accuracy: 76.08%, Train loss: 0.3819, Val loss: 0.5239
Epoch: 8/20, Accuracy: 73.39%, Train loss: 0.3520, Val loss: 0.6106
Epoch: 9/20, Accuracy: 80.07%, Train loss: 0.3270, Val loss: 0.4329
Epoch: 10/20, Accuracy: 74.45%, Train loss: 0.3048, Val loss: 0.5532
Epoch: 11/20, Accuracy: 79.33%, Train loss: 0.2875, Val loss: 0.4635
Epoch: 12/20, Accuracy: 82.46%, Train loss: 0.2692, Val loss: 0.3213
Epoch: 13/20, Accuracy: 81.39%, Train loss: 0.2604, Val loss: 0.3654
Epoch: 14/20, Accuracy: 83.33%, Train loss: 0.2472, Val loss: 0.2869
Epoch: 15/20, Accuracy: 84.46%, Train loss:

In [6]:
loss, accuracy = model.evaluate(test_loader)
print(f" Accuracy: {100*accuracy:.2f}%")

 Accuracy: 87.20%


In [8]:
model.save()

Checkpoint saved at /home/jorgetf/testmodel/Network-Packet-ML-Model/large_models/checkpoint/mlp_4.pth


In [7]:
"""
def plot_data(data, type, title, n_epochs=conf["epochs"]):
    epochs = [epoch for epoch in range(1, n_epochs + 1)]
    data = [d.to('cpu') for d in data]
    plt.plot(epochs, data, marker="+", linestyle="dotted")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel(type)
    plt.show()
"""

'\ndef plot_data(data, type, title, n_epochs=conf["epochs"]):\n    epochs = [epoch for epoch in range(1, n_epochs + 1)]\n    data = [d.to(\'cpu\') for d in data]\n    plt.plot(epochs, data, marker="+", linestyle="dotted")\n    plt.title(title)\n    plt.xlabel("Epochs")\n    plt.ylabel(type)\n    plt.show()\n'